In [4]:
#tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper


In [11]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name


'wikipedia'

In [14]:
api_warapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250 )
arxiv = ArxivQueryRun(api_wrapper=api_warapper_arxiv)
arxiv.name

'arxiv'

In [15]:
tools=[wiki, arxiv]

In [17]:
#custom tool---rag tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [19]:
#load the page -- website
loader = WebBaseLoader('https://learn.microsoft.com/en-us/intune/intune-service/fundamentals/whats-new')
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100).split_documents(docs)
vectordb = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023887B8F890>, search_kwargs={})

In [20]:
#convert reriever to tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "IntuneDoumentation", "Search any info about Intune")
retriever_tool.name

'IntuneDoumentation'

In [21]:
tools = [wiki, arxiv, retriever_tool]

In [ ]:
#run all tools with agents and llm models-agent executor
import os
import openai
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

llm= ChatGroq(groq_api_key=groq_api_key, model_name = "Llama3-8b-8192")


In [27]:
#instead chatprompt temp use this also
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [30]:
#agent executor
from langchain.agents import AgentExecutor
agent_executor= AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [31]:
agent_executor.invoke({"input":"Tell me about Intune"})



> Entering new AgentExecutor chain...

Invoking: `IntuneDoumentation` with `{'query': 'Intune'}`


For more information about EPM, which is available as an Intune Suite add-on-capability, see Endpoint Privilege Management overview.
App management
Newly available protected apps for Intune
The following protected apps are now available for Microsoft Intune:

Now when users select I own this device during a BYOD enrollment, Microsoft Intune enrolls them via account-driven user enrollment, rather than profile-based user enrollment, and then secures only work-related apps. Less than one percent of Apple devices across all Intune tenants are currently enrolled this way, so this change doesn't affect most enrolled devices. There is no change for iOS users who select My company owns this device during a BYOD enrollment. Intune enrolls them via device enrollment with Intune Company Portal, and then secures their entire device.
If you currently allow users in BYOD scenarios to determine their 

{'input': 'Tell me about Intune',
 'output': 'It seems that you\'ve called the "IntuneDoumentation" tool and received some information about Intune. Based on the output, it appears that Intune is a suite of tools provided by Microsoft that helps organizations manage and secure their devices and applications. The output mentions several features and updates, such as account-driven user enrollment, device inventory, app management, and device configuration.\n\nTo provide more insight, Intune is a cloud-based service that allows administrators to manage and monitor their devices, including Windows, iOS, and Android devices. It provides a range of features, including device enrollment, app management, and security policies. Intune is part of the Microsoft Enterprise Mobility + Security (EMS) suite and is designed to help organizations protect their data and applications from endpoint threats.\n\nIf you have any specific questions or would like me to elaborate on any of the features mention